# Sparse Reconstruction Localization Analysis

This notebook demonstrates the **joint sparse superposition reconstruction** approach for transmitter localization:
1. Load processed power measurements from monitoring locations
2. Convert data from dB to linear scale
3. Run sparse reconstruction algorithm
4. Visualize sparse transmit power field
5. Compare with likelihood-based approach

---

## Overview

**Sparse Reconstruction Formulation:**

$$\hat{\mathbf{t}} = \arg\min_{\mathbf{t}\ge 0} \|\mathbf{W}(\log_{10}(\mathbf{A}_{\text{model}}\mathbf{t}) - \log_{10}(\mathbf{p}))\|_{2}^{2} + \lambda \|\mathbf{t}\|_{1}$$

**Key Features:**
- Single-stage joint optimization (vs. two-stage likelihood)
- Explicit sparsity constraint (few active transmitters)
- Convex optimization (global optimum guaranteed)
- Fast computation (~10-30 seconds vs. 5-10 minutes)

---

## Step 1: Setup and Configuration

Import modules and configure the analysis.

In [ ]:
# Import necessary modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yaml
import sys
import time
from pathlib import Path

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent))

# Import utility functions
from src.utils import (
    load_slc_map, 
    load_monitoring_locations, 
    get_sensor_locations_array,
    load_transmitter_locations,
    lin_to_dB
)

# Import SPARSE RECONSTRUCTION modules
from src.sparse_reconstruction import (
    joint_sparse_reconstruction,
    compute_signal_strength_at_points,
    dbm_to_linear,
    linear_to_dbm
)

# Import visualization
from src.visualization.spatial_plots import (
    plot_transmit_power_map,
    plot_signal_estimates_map
)

print("✓ Modules imported successfully")
print("✓ Using SPARSE RECONSTRUCTION algorithm")

In [ ]:
'''
from itertools import combinations
import numpy as np
import yaml
import sys
import time
from pathlib import Path

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent))

# Import utility functions
from src.utils import (
    load_slc_map, 
    load_monitoring_locations, 
    get_sensor_locations_array,
    load_transmitter_locations
)

# Import SPARSE RECONSTRUCTION modules
from src.sparse_reconstruction import joint_sparse_reconstruction

# ============================================================================
# CONFIGURATION
# ============================================================================

ALL_TRANSMITTERS = ['mario', 'moran', 'guesthouse', 'wasatch', 'ustar']
SEEDS = [39, 45, 52, 55, 58, 61, 64, 70, 73, 123]

# Generate all subsets of size 1, 2, 3, 4, and 5
TRANSMITTER_SUBSETS = []
for size in [1, 2, 3, 4, 5]:
    TRANSMITTER_SUBSETS.extend([list(combo) for combo in combinations(ALL_TRANSMITTERS, size)])

print(f"Total configurations: {len(SEEDS) * len(TRANSMITTER_SUBSETS)}")

# ============================================================================
# LOAD MAP AND CONFIG (one-time load, shared across all iterations)
# ============================================================================

with open('../config/parameters.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("Loading SLC map...")
map_data = load_slc_map(
    map_folder_dir="../",
    downsample_factor=config['spatial']['downsample_factor']
)
print(f"✓ Map loaded: shape {map_data['shape']}")

# Load all transmitter locations (for metrics/validation later)
all_tx_locations = load_transmitter_locations(
    config_path='../config/transmitter_locations.yaml',
    map_data=map_data
)

# ============================================================================
# MAIN LOOP
# ============================================================================

failed = []
total = len(SEEDS) * len(TRANSMITTER_SUBSETS)

for i, (random_seed, transmitters) in enumerate([(s, t) for s in SEEDS for t in TRANSMITTER_SUBSETS], 1):
    TRANSMITTER_COMMA = ",".join(transmitters)
    TRANSMITTER_UNDERSCORE = "_".join(transmitters)
    
    print(f"\n{'='*70}")
    print(f"[{i}/{total}] Processing: {TRANSMITTER_COMMA} | seed={random_seed}")
    print(f"{'='*70}")
    
    try:
        # ---------------------------------------------------------------
        # Step 1: Process raw data to monitoring locations
        # ---------------------------------------------------------------
        !python ../scripts/process_raw_data_to_monitoring.py \
            --input-dir "C:/Users/serha/raw_data/stat_rot/stat/" \
            --transmitter {TRANSMITTER_COMMA} \
            --num-locations 10 \
            --output-yaml ../config/monitoring_locations_{TRANSMITTER_UNDERSCORE}_seed_{random_seed}.yaml \
            --output-data ../data/processed/{TRANSMITTER_UNDERSCORE}_seed_{random_seed}/ \
            --dedup-threshold 5.0 \
            --random-seed {random_seed}
        
        # ---------------------------------------------------------------
        # Step 2: Load processed data
        # ---------------------------------------------------------------
        CUSTOM_YAML = f'../config/monitoring_locations_{TRANSMITTER_UNDERSCORE}_seed_{random_seed}.yaml'
        DATA_DIR = Path(f'../data/processed/{TRANSMITTER_UNDERSCORE}_seed_{random_seed}/')
        
        locations_config = load_monitoring_locations(
            config_path=CUSTOM_YAML,
            map_data=map_data
        )
        
        observed_powers_dB = np.load(DATA_DIR / f"{TRANSMITTER_UNDERSCORE}_avg_powers.npy")
        sensor_locations = get_sensor_locations_array(locations_config)
        
        # ---------------------------------------------------------------
        # Step 3: Run joint_sparse_reconstruction with TIREM to cache
        # ---------------------------------------------------------------
        print(f"Running TIREM-based joint_sparse_reconstruction to cache propagation data...")
        
        start_time = time.time()
        
        tx_map, info = joint_sparse_reconstruction(
            sensor_locations=sensor_locations,
            observed_powers_dBm=observed_powers_dB,
            input_is_linear=False,
            solve_in_linear_domain=True,
            map_shape=map_data['shape'],
            scale=config['spatial']['proxel_size'],
            np_exponent=config['localization']['path_loss_exponent'],
            lambda_reg=0,
            norm_exponent=0,
            enable_reweighting=False,
            whitening_method='hetero_geo_aware',
            sigma_noise=5e-9,
            eta=0.01,
            feature_rho=[0.2, 1e10, 1e10, 1e10],
            solver='glrt',
            selection_method='max',
            cluster_max_candidates=30,
            glrt_max_iter=30,
            glrt_threshold=4.0,
            dedupe_distance_m=25.0,
            return_linear_scale=False,
            verbose=True,
            model_type='tirem',
            model_config_path='../config/tirem_parameters.yaml',
            n_jobs=-1
        )
        
        elapsed_time = time.time() - start_time
        print(f"✓ Completed in {elapsed_time:.2f}s | Transmitters found: {len(info.get('solver_info', {}).get('support', []))}")
        
    except Exception as e:
        print(f"FAILED: {TRANSMITTER_UNDERSCORE} seed={random_seed} - {e}")
        failed.append((TRANSMITTER_UNDERSCORE, random_seed, str(e)))

# ============================================================================
# SUMMARY
# ============================================================================

print(f"\n{'='*70}")
print(f"BATCH COMPLETED!")
print(f"{'='*70}")
print(f"Successful: {total - len(failed)}/{total}")
print(f"Failed: {len(failed)}/{total}")

if failed:
    print("\nFailed configurations:")
    for tx, seed, err in failed:
        print(f"  - {tx} (seed={seed}): {err[:50]}...")

'''

In [ ]:
# Configure which transmitter(s) to analyze
TRANSMITTERS = ['mario', 'moran']
TRANSMITTER_COMMA = ",".join(TRANSMITTERS)
TRANSMITTER_UNDERSCORE = "_".join(TRANSMITTERS)

print(f"Analyzing transmitter(s): {TRANSMITTER_COMMA}")

random_seed=32

'''
if random_seed is not None:
    !python ../scripts/process_raw_data_to_monitoring.py --input-dir "C:/Users/serha/raw_data/stat_rot/stat/" --transmitter {TRANSMITTER_COMMA} --num-locations 10 --output-yaml ../config/monitoring_locations_{TRANSMITTER_UNDERSCORE}_seed_{random_seed}.yaml --output-data ../data/processed/{TRANSMITTER_UNDERSCORE}_seed_{random_seed}/ --dedup-threshold 5.0 --random-seed {random_seed}
else:
    !python ../scripts/process_raw_data_to_monitoring.py --input-dir "C:/Users/serha/raw_data/stat_rot/stat/" --transmitter {TRANSMITTER_COMMA} --num-locations 10 --output-yaml ../config/monitoring_locations_{TRANSMITTER_UNDERSCORE}.yaml --output-data ../data/processed/{TRANSMITTER_UNDERSCORE}/ --dedup-threshold 5.0
'''

In [ ]:
# Set file paths
if random_seed is not None:
    CUSTOM_YAML = f'../config/monitoring_locations_{TRANSMITTER_UNDERSCORE}_seed_{random_seed}.yaml'
    DATA_DIR = Path(f'../data/processed/{TRANSMITTER_UNDERSCORE}_seed_{random_seed}/')
else:
    CUSTOM_YAML = f'../config/monitoring_locations_{TRANSMITTER_UNDERSCORE}.yaml'
    DATA_DIR = Path(f'../data/processed/{TRANSMITTER_UNDERSCORE}/')

print(f"Configuration file: {CUSTOM_YAML}")
print(f"Data directory: {DATA_DIR}")

## Step 2: Load Map and Configuration

In [ ]:
# Load configuration
with open('../config/parameters.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("Configuration loaded:")
print(f"  Proxel size: {config['spatial']['proxel_size']} m/pixel")
print(f"  Path loss exponent: {config['localization']['path_loss_exponent']}")
print(f"  Shadowing σ: {config['localization']['std_deviation']} dB")
print(f"  Correlation distance δ_c: {config['localization']['correlation_coeff']} m")

In [ ]:
# Load SLC map
print("Loading SLC map...")
map_data = load_slc_map(
    map_folder_dir="../",
    downsample_factor=config['spatial']['downsample_factor']
)

print(f"✓ Map loaded: shape {map_data['shape']}")
print(f"  UTM Easting range: [{map_data['UTM_long'].min():.1f}, {map_data['UTM_long'].max():.1f}] m")
print(f"  UTM Northing range: [{map_data['UTM_lat'].min():.1f}, {map_data['UTM_lat'].max():.1f}] m")

In [ ]:
# Load transmitter locations for visualization
print("Loading transmitter locations...")
all_tx_locations = load_transmitter_locations(
    config_path='../config/transmitter_locations.yaml',
    map_data=map_data
)

# Filter to only show transmitters being analyzed
tx_locations = {name: all_tx_locations[name] for name in TRANSMITTERS if name in all_tx_locations}

print(f"✓ Loaded {len(tx_locations)} transmitter location(s) for display")
print(f"\nTransmitter Locations:")
for tx_name, tx_data in tx_locations.items():
    print(f"  {tx_name:12s}: ({tx_data['latitude']:.6f}, {tx_data['longitude']:.6f}) → {tx_data['coordinates']}")

## Step 3: Load Monitoring Locations and Power Measurements

In [ ]:
# Load monitoring locations
print(f"Loading monitoring locations from {CUSTOM_YAML}...")
locations_config = load_monitoring_locations(
    config_path=CUSTOM_YAML,
    map_data=map_data
)

print(f"✓ Loaded {len(locations_config['data_points'])} monitoring locations")
print(f"  UTM Zone: {locations_config['utm_zone']}{'N' if locations_config['northern_hemisphere'] else 'S'}")

# Display locations
print("\nMonitoring Locations:")
for loc in locations_config['data_points']:
    print(f"  {loc['name']:12s}: ({loc['latitude']:.5f}, {loc['longitude']:.5f}) → {loc['coordinates']}")

In [ ]:
# Load processed power measurements (in dB)
print(f"\nLoading power measurements from {DATA_DIR}...")

observed_powers_dB = np.load(DATA_DIR / f"{TRANSMITTER_UNDERSCORE}_avg_powers.npy")
power_stds = np.load(DATA_DIR / f"{TRANSMITTER_UNDERSCORE}_std_powers.npy")
sample_counts = np.load(DATA_DIR / f"{TRANSMITTER_UNDERSCORE}_sample_counts.npy")

print(f"✓ Loaded power measurements:")
print(f"  Number of sensors: {len(observed_powers_dB)}")
print(f"  Power range: [{observed_powers_dB.min():.2f}, {observed_powers_dB.max():.2f}] dB")
print(f"  Total samples: {sample_counts.sum()}")
print(f"  Observed powers (dB): {observed_powers_dB}")

In [ ]:
# Display summary table
summary_df = pd.read_csv(DATA_DIR / f"{TRANSMITTER_UNDERSCORE}_summary.csv")
print("\nPower Measurement Summary:")
print(summary_df.to_string(index=False))

## Step 4: Convert Data for Sparse Reconstruction

The sparse reconstruction algorithm works in **linear power scale** (mW), not dB.

**Conversion:** $P[\text{mW}] = 10^{P[\text{dBm}]/10}$

In [ ]:
# Convert observed powers from dBm to linear scale (mW)
observed_powers_linear = dbm_to_linear(observed_powers_dB)

print("Unit Conversion:")
print(f"  Input (dBm):  {observed_powers_dB}")
print(f"  Output (mW):  {observed_powers_linear}")
print(f"\n  Linear scale range: [{observed_powers_linear.min():.2e}, {observed_powers_linear.max():.2e}] mW")

# Note: These are very small numbers (e.g., 1e-8 mW = 1e-11 W = 10 pW)
# This is normal for RF propagation at ~100m distances

In [ ]:
# Get sensor locations in pixel coordinates
sensor_locations = get_sensor_locations_array(locations_config)

print(f"Sensor locations (pixel coordinates):")
print(f"  Shape: {sensor_locations.shape}")
print(f"  Array: {sensor_locations}")

## Step 5: Run Sparse Reconstruction Algorithm

**Optimization Problem:**

$$\hat{\mathbf{t}} = \arg\min_{\mathbf{t}\ge 0} \|\mathbf{W}(\log_{10}(\mathbf{A}_{\text{model}}\mathbf{t}) - \log_{10}(\mathbf{p}))\|_{2}^{2} + \lambda \|\mathbf{t}\|_{1}$$

**Parameters:**
- $\mathbf{t}$: Transmit power field (N grid points)
- $\mathbf{p}$: Observed powers (M sensors)
- $\mathbf{A}_{\text{model}}$: Propagation matrix (linear path gains)
- $\mathbf{W}$: Whitening matrix ($\mathbf{V}^{-1/2}$)
- $\lambda$: Sparsity regularization parameter

In [ ]:
mean_power = np.mean(observed_powers_linear)
#lambda_reg = 50 / mean_power
lambda_reg = 0
print(f"Regularization parameter: λ = {lambda_reg:.2e}")

# Run sparse reconstruction
print("\n" + "="*70)
print("RUNNING SPARSE RECONSTRUCTION ALGORITHM")
print("="*70)

start_time = time.time()

tx_map_sparse_reweighted, info_sparse_reweighted = joint_sparse_reconstruction(
    sensor_locations=sensor_locations,
    observed_powers_dBm=observed_powers_dB,  # Function converts internally
    input_is_linear=False,
    solve_in_linear_domain=True,
    map_shape=map_data['shape'],
    scale=config['spatial']['proxel_size'],
    np_exponent=config['localization']['path_loss_exponent'],
    lambda_reg=lambda_reg,
    norm_exponent=0,
    enable_reweighting=False, # Set to True to enable reweighting
    whitening_method='hetero_geo_aware', # 'hetero_diag' or 'spatial_corr_exp_decay' or 'hetero_geo_aware'
    sigma_noise=5e-9, # The standard deviation of the noise
    eta=0.01, # The weight of the reweighting term
    feature_rho=[0.2, 1e10, 1e10, 1e10],  # Custom norm. values for LOS, shadowing, obstacles, distance
    solver='glrt',  # Use scipy L-BFGS-B for non-convex log-domain objective
    selection_method='max',  # Use cluster selection method, 'max' or 'cluster'
    cluster_max_candidates=30,  # The maximum number of candidates to consider for clustering
    glrt_max_iter=30,  # The maximum number of iterations for the GLRT algorithm
    glrt_threshold=4.0,   # The threshold for the GLRT test statistic
    dedupe_distance_m=25.0,  # The deduplication distance for transmitters placed too close to each other
    return_linear_scale=False,  # Return in dBm for visualization
    verbose=True,
    model_type='raytracing',
    model_config_path='../config/sionna_parameters.yaml',
    n_jobs=-1 
)

elapsed_time = time.time() - start_time

print(f"\n✓ Reconstruction completed in {elapsed_time:.2f} seconds")

## Step 6: Analyze Sparse Solution

In [ ]:
# Find top-K non-zero locations
K = 10  # Number of top locations to display

# Convert to linear scale for sorting
tx_map_linear_reweighted = dbm_to_linear(tx_map_sparse_reweighted)
flat_powers_reweighted = tx_map_linear_reweighted.ravel()
top_k_indices_reweighted = np.argsort(flat_powers_reweighted)[-K:][::-1]  # Top K descending

print(f"\nTop {K} Transmitter Locations (by power):")
print("-" * 80)
print(f"{'Rank':<6} {'Row':<8} {'Col':<8} {'Power (dBm)':<15} {'Power (mW)':<15}")
print("-" * 80)

for rank, idx in enumerate(top_k_indices_reweighted, 1):
    row = idx // map_data['shape'][1]
    col = idx % map_data['shape'][1]
    power_linear = flat_powers_reweighted[idx]
    power_dBm = linear_to_dbm(power_linear)
    
    if power_linear > 1e-15:  # Only show non-negligible values
        print(f"{rank:<6} {row:<8} {col:<8} {power_dBm:<15.2f} {power_linear:<15.2e}")

print("-" * 80)

## Step 7: Visualize Sparse Transmit Power Map

In [ ]:
# Visualize sparse transmit power field
tx_name_list = " & ".join(TRANSMITTERS)

fig, ax = plot_transmit_power_map(
    transmit_power_map=tx_map_sparse_reweighted,
    data_points=sensor_locations,
    observed_powers=observed_powers_dB,
    UTM_lat=map_data['UTM_lat'],
    UTM_long=map_data['UTM_long'],
    band_name=f"{tx_name_list} Transmitter",
    transmitter_locations=tx_locations
)

# Add sparsity annotation
ax.text(
    0.02, 0.98, 
    f"Sparsity: {info_sparse_reweighted['sparsity']*100:.1f}%\n"
    f"Non-zero: {info_sparse_reweighted['n_nonzero']} / {np.prod(map_data['shape'])}\n"
    f"λ = {lambda_reg:.2e}",
    transform=ax.transAxes,
    fontsize=10,
    verticalalignment='top',
    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8)
)

plt.tight_layout()
plt.show()

In [ ]:
# Check the indices of the 3 found transmitters
support_indices = info_sparse_reweighted['solver_info']['support']
height, width = map_data['shape']

print(f"Found {len(support_indices)} transmitters at indices: {support_indices}")

for idx in support_indices:
    r, c = divmod(idx, width)
    power = tx_map_sparse_reweighted[r, c]
    print(f"  - Location (row={r}, col={c}), Power={power:.2f} dBm")

## Step 7.5: Visualize GLRT Candidates History

If the GLRT solver was used, we can visualize the candidates considered at each iteration.

In [ ]:
# Visualize GLRT Iteration History
if 'solver_info' in info_sparse_reweighted and 'candidates_history' in info_sparse_reweighted['solver_info']:
    solver_info = info_sparse_reweighted['solver_info']
    history = solver_info['candidates_history']
    print(f"\nVisualizing candidates at each iteration... ({len(history)} iterations)")
    
    # Helper to plot map with UTM coordinates
    def plot_glrt_step(score_map, iter_num, selected_idx, selected_score, score_label="GLRT Score"):
        fig = plt.figure(figsize=(13, 8))
        ax = fig.gca()
        
        # Plot sensors
        scatter = ax.scatter(sensor_locations[:, 0], sensor_locations[:, 1],
                             c=observed_powers_dB, s=150, edgecolor='green',
                             linewidth=2, cmap='hot', label='Monitoring Locations', zorder=6)
        
        tx_locs = globals().get('tx_locations', None)
        if tx_locs is not None:
            tx_coords = np.array([tx['coordinates'] for tx in tx_locs.values()])
            ax.scatter(tx_coords[:, 0], tx_coords[:, 1],
                       marker='x', s=200, c='blue', linewidth=3,
                       label='True Transmitter Locations', zorder=10)
        
        # Plot candidates (sparse score map)
        nonzero_mask = score_map > 0
        if np.sum(nonzero_mask) > 0:
            nonzero_indices = np.argwhere(nonzero_mask)
            nonzero_row = nonzero_indices[:, 0]
            nonzero_col = nonzero_indices[:, 1]
            nonzero_values = score_map[nonzero_mask]
            
            sparse_scatter = ax.scatter(nonzero_col, nonzero_row,
                                      c=nonzero_values, s=300, marker='s',
                                      cmap='viridis', edgecolor='black', linewidth=1,
                                      label='Candidates', zorder=5)
            
            cbar = plt.colorbar(sparse_scatter, label=score_label)
            cbar.ax.tick_params(labelsize=18)
            cbar.set_label(label=score_label, size=18)
            cbar.ax.set_position([0.77, 0.1, 0.04, 0.8])
            
        # Highlight selected
        sel_row, sel_col = np.unravel_index(selected_idx, map_data['shape'])
        ax.scatter([sel_col], [sel_row], c='magenta', marker='*', s=400, label='Selected Candidate', zorder=11)

        UTM_lat = map_data['UTM_lat']
        UTM_long = map_data['UTM_long']
        interval = max(1, len(UTM_lat) // 5)
        tick_values = list(range(0, len(UTM_lat), interval))
        tick_labels = ['{:.1f}'.format(lat) for lat in UTM_lat[::interval]]
        plt.xticks(ticks=tick_values, labels=tick_labels, fontsize=14, rotation=0)

        interval = max(1, len(UTM_long) // 5)
        tick_values = list(range(0, len(UTM_long), interval))
        tick_labels = ['{:.1f}'.format(lat) for lat in UTM_long[::interval]]
        plt.yticks(ticks=[0] + tick_values[1:], labels=[""] + tick_labels[1:], fontsize=14, rotation=90)

        ax.set_xlim([0, map_data['shape'][1]])
        ax.set_ylim([0, map_data['shape'][0]])
        
        plt.xlabel('UTM$_E$ [m]', fontsize=18, labelpad=10)
        plt.ylabel('UTM$_N$ [m]', fontsize=18, labelpad=10)
        
        # Dynamic Title
        plt.title(f"GLRT Iteration {iter_num} ({score_label}: {selected_score:.4f})", fontsize=20)
        
        scatter_cbar = plt.colorbar(scatter, label='Observed Signal [dBm]', location='left')
        scatter_cbar.ax.tick_params(labelsize=18)
        scatter_cbar.set_label(label='Observed Signal [dBm]', size=18)
        scatter_cbar.ax.set_position([0.18, 0.1, 0.04, 0.8])
        
        plt.legend(loc='upper right')
        plt.show()

    # Determine what to show
    whitening_method = solver_info.get('whitening_method', 'unknown')
    
    for item in history:
        height, width = map_data['shape']
        score_map = np.zeros((height, width))
        
        top_indices = item['top_indices']
        top_scores = item['top_scores']
        
        # Fill scores (raw scores are usually plotted on map for contrast)
        rows, cols = np.unravel_index(top_indices, (height, width))
        score_map[rows, cols] = top_scores
        
        # Use normalized score for title if available and relevant
        if whitening_method == 'hetero_geo_aware':
             display_score = item.get('normalized_score', item['selected_score'])
             label = "Corrected Score"
        else:
             display_score = item['selected_score']
             label = "GLRT Score"
        
        plot_glrt_step(score_map, item['iteration'], item['selected_index'], display_score, score_label=label)

## Step 8: Localization Metrics

In [ ]:
# Import metrics module
from src.evaluation.metrics import compute_localization_metrics, extract_locations_from_map

# ============================================================================
# METRICS CALCULATION
# ============================================================================

print("\n" + "="*70)
print("LOCALIZATION METRICS")
print("="*70)

# 1. Get True Locations
# tx_locations is a dict {name: data}. data['coordinates'] is [col, row]
true_locs_pixels = np.array([tx['coordinates'] for tx in tx_locations.values()])

# 2. Get Estimated Locations
# From info_sparse_reweighted['solver_info']['support'] if available (GLRT)
if 'solver_info' in info_sparse_reweighted and 'support' in info_sparse_reweighted['solver_info']:
    support_indices = info_sparse_reweighted['solver_info']['support']
    height, width = map_data['shape']
    
    # Filter out transmitters with power below -190 dBm
    valid_indices = []
    for idx in support_indices:
        r, c = idx // width, idx % width
        power_dbm = tx_map_sparse_reweighted[r, c]
        if power_dbm > -190:
            valid_indices.append(idx)
    
    print(f"Filtered: {len(support_indices)} -> {len(valid_indices)} transmitters (removed {len(support_indices) - len(valid_indices)} with power < -190 dBm)")
    
    est_rows = [idx // width for idx in valid_indices]
    est_cols = [idx % width for idx in valid_indices]
    est_locs_pixels = np.column_stack((est_cols, est_rows)) if valid_indices else np.empty((0, 2))
else:
    # Fallback: extract from map
    est_locs_pixels = extract_locations_from_map(tx_map_sparse_reweighted, threshold=1e-10)

# 3. Compute Metrics
metrics = compute_localization_metrics(
    true_locations=true_locs_pixels,
    estimated_locations=est_locs_pixels,
    scale=config['spatial']['proxel_size'],
    tolerance=200.0  # meters
)

# 4. Display Results
print(f"True Transmitters:      {metrics['n_true']}")
print(f"Estimated Transmitters: {metrics['n_est']}")

if not np.isnan(metrics['ale']):
    print(f"\nAverage Localization Error (ALE): {metrics['ale']:.2f} meters")
else:
    print("\nAverage Localization Error (ALE): N/A")

print(f"\nDetection Metrics (Tolerance = 200.0 m):")
print(f"  True Positives (TP):  {metrics['tp']}")
print(f"  False Positives (FP): {metrics['fp']}")
print(f"  False Negatives (FN): {metrics['fn']}")
print(f"  Probability of Detection (Pd): {metrics['pd']*100:.1f}%")
print(f"  Precision:                     {metrics['precision']*100:.1f}%")
print(f"  False Alarm Rate (1-Precision): {metrics['far']*100:.1f}%")
print(f"  F1 Score:                      {metrics['f1_score']:.4f}")

## Step 9: Compare Predictions with Observations

In [ ]:
'''
# Compute predicted powers at sensor locations using sparse transmit field
print("Computing signal strength at sensor locations...")

# Need to convert tx_map back to linear for physics-based propagation
tx_map_linear = dbm_to_linear(tx_map_sparse)

predicted_powers_linear = compute_signal_strength_at_points(
    transmit_power_map_linear=tx_map_linear,
    target_locations=sensor_locations,
    scale=config['spatial']['proxel_size'],
    np_exponent=config['localization']['path_loss_exponent'],
    return_linear_scale=False,  # Return in dBm
    verbose=True
)

predicted_powers_dB = predicted_powers_linear

print(f"\n✓ Predictions computed")
'''

In [ ]:
'''
# Evaluation metrics
print("\n" + "="*70)
print("EVALUATION METRICS")
print("="*70)

# Per-sensor comparison
print(f"\nPer-Sensor Comparison:")
print("-" * 80)
print(f"{'Location':<15} {'Observed (dBm)':<18} {'Predicted (dBm)':<18} {'Error (dB)':<12}")
print("-" * 80)

errors = []
for i, loc in enumerate(locations_config['data_points']):
    obs = observed_powers_dB[i]
    pred = predicted_powers_dB[i]
    error = obs - pred
    errors.append(error)
    print(f"{loc['name']:<15} {obs:<18.2f} {pred:<18.2f} {error:<12.2f}")

errors = np.array(errors)
print("-" * 80)

# Summary statistics
mse = np.mean(errors**2)
rmse = np.sqrt(mse)
mae = np.mean(np.abs(errors))
max_error = np.max(np.abs(errors))

print(f"\nSummary Statistics:")
print(f"  Mean Squared Error (MSE):  {mse:.2f} dB²")
print(f"  Root Mean Squared Error:   {rmse:.2f} dB")
print(f"  Mean Absolute Error (MAE): {mae:.2f} dB")
print(f"  Max Absolute Error:        {max_error:.2f} dB")

# Compare with baseline (mean predictor)
variance_baseline = np.var(observed_powers_dB)
variance_reduction = (1 - mse / variance_baseline) * 100

print(f"\n  Baseline variance (mean predictor): {variance_baseline:.2f} dB²")
print(f"  Variance reduction: {variance_reduction:.1f}%")

if variance_reduction > 0:
    print(f"  ✓ Model outperforms baseline predictor")
else:
    print(f"  ✗ Model underperforms baseline (consider adjusting λ)")

print("="*70)
'''

In [ ]:
'''
# Scatter plot: Observed vs Predicted
fig, ax = plt.subplots(figsize=(8, 8))

ax.scatter(observed_powers_dB, predicted_powers_dB, s=100, alpha=0.7, edgecolors='k')

# Add diagonal line (perfect prediction)
min_val = min(observed_powers_dB.min(), predicted_powers_dB.min())
max_val = max(observed_powers_dB.max(), predicted_powers_dB.max())
ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')

ax.set_xlabel('Observed Power (dBm)', fontsize=12)
ax.set_ylabel('Predicted Power (dBm)', fontsize=12)
ax.set_title('Sparse Reconstruction: Observed vs Predicted Power', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Add statistics text box
stats_text = f"RMSE = {rmse:.2f} dB\nMAE = {mae:.2f} dB\nSparsity = {info_sparse['sparsity']*100:.1f}%"
ax.text(
    0.05, 0.95, stats_text,
    transform=ax.transAxes,
    fontsize=10,
    verticalalignment='top',
    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8)
)

plt.tight_layout()
plt.show()
'''